In [ ]:
from __future__ import print_function
import numpy as np
from matplotlib import pyplot as plt
import cv2
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import theano

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3
nb_classes = 10
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_tmp_test = X_test 

model = Sequential()
model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))# Layer 1
model.add(Activation('relu'))# Layer 2
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))# Layer 3
model.add(Activation('relu'))# Layer 4
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))# Layer 5
model.add(Dropout(0.25))# Layer 6
model.add(Flatten())# Layer 7
model.add(Dense(128))# Layer 8
model.add(Activation('relu'))# Layer 9
model.add(Dropout(0.5))# Layer 10
model.add(Dense(nb_classes))# Layer 11
model.add(Activation('softmax')) # 12
model.compile(loss='categorical_crossentropy', optimizer='adadelta')
model.load_weights('mnist.hdf5')

# Create an image from test set and get its feature maps

im= X_tmp_test[12]
'''
new = np.zeros((28,28))
im = im[6:26,4:24]
end_row = -1
end_col=-1
for row in range(0,im.shape[0]):
    end_row = end_row +1
    end_col=-1
    for col in range(3,im.shape[0]):
        end_col= end_col +1
        new[end_row][end_col] = im[row][col] 
#  Print that image'''
smallnew = cv2.resize(im, (0,0), fx=0.5, fy=0.5)
tmp = np.zeros((28,28))
end_row = -1
end_col=-1
for row in range(0,smallnew.shape[0]):
    end_row = end_row +1
    end_col=-1
    for col in range(3,smallnew.shape[0]):
        end_col= end_col +1
        tmp[end_row][end_col] = smallnew[row][col] 
plt.imshow(tmp, interpolation='nearest')
plt.show()

#  Predict that image
sample =[]# From vectors to algo readable format
sample.append(tmp)
sample = np.array(sample)
sample= sample.reshape(sample.shape[0], 1, img_rows, img_cols)
out = model.predict(sample)
print(out)
# Then get its feature map for the last convolution layer

'''for fmap in range(0,32):
    get_activations = theano.function([model.layers[0].input], model.layers[5].get_output(train=False), allow_input_downcast=True)
    activations = get_activations(sample)
    print(activations.shape, activations[0][fmap])
    # plt.imshow(activations[0][fmap], interpolation='nearest')
    # plt.show()'''
# Then get the bounding box by moving backwards
get_activations = theano.function([model.layers[0].input], model.layers[5].get_output(train=False), allow_input_downcast=True)
activations = get_activations(sample)
# Getting boundaries on the last convolution layer
bmap1 = np.zeros((24,24))
for fmap in range(0,32):
    for rows in range(0,12):
        for cols in range(0,12):
            if activations[0][fmap][rows][cols] > 1:
                bmap1[rows*2][cols*2] = 1
                bmap1[rows*2][cols*2+1] = 1
                bmap1[rows*2+1][cols*2] = 1
                bmap1[rows*2+1][cols*2+1] = 1
# Getting boundaries on the first convolution layer
bmap2 = np.zeros((26,26))
for rows in range(0,24):
    for cols in range(0,24):
        if bmap1[rows][cols] ==1:
            bmap2[rows][cols] = 1
            bmap2[rows][cols+1] = 1
            bmap2[rows][cols+2] = 1
            bmap2[rows+1][cols] = 1
            bmap2[rows+1][cols+1] = 1
            bmap2[rows+1][cols+2] = 1
            bmap2[rows+2][cols] = 1
            bmap2[rows+2][cols+1] = 1
            bmap2[rows+2][cols+2] = 1

# Getting boundaries on the input layer
bmap3 = np.zeros((28,28))
rows_list=[]
cols_list=[]
for rows in range(0,26):
    for cols in range(0,26):
        if bmap2[rows][cols] ==1:
            bmap3[rows][cols] = 1
            bmap3[rows][cols+1] = 1
            bmap3[rows][cols+2] = 1
            bmap3[rows+1][cols] = 1
            bmap3[rows+1][cols+1] = 1
            bmap3[rows+1][cols+2] = 1
            bmap3[rows+2][cols] = 1
            bmap3[rows+2][cols+1] = 1
            bmap3[rows+2][cols+2] = 1
            rows_list.append(rows)
            rows_list.append(rows+1)
            rows_list.append(rows+2)
            cols_list.append(cols)
            cols_list.append(cols+1)
            cols_list.append(cols+2)
row_min = min(rows_list)
row_max= max(rows_list)
col_min = min(cols_list)
col_max= max(cols_list)
for cols in range(col_min, col_max+1):
    tmp[row_min][cols] = 100
    tmp[row_max][cols] = 100
for rows in range(row_min, row_max+1):
    tmp[rows][col_min] = 100
    tmp[rows][col_max] = 100
plt.imshow(tmp, interpolation='nearest')
plt.show()

[[  0.00000000e+00   2.19752252e-07   0.00000000e+00   4.53017009e-33
    9.99999762e-01   0.00000000e+00   3.41157353e-28   2.35640374e-09
    0.00000000e+00   0.00000000e+00]]
